<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Moving_Average_Crossover_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance numpy pandas matplotlib plotly mplfinance
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import mplfinance as mpf

# Ensure that our plots are displayed inline within the Jupyter Notebook
%matplotlib inline
# Define the ticker symbols for the assets we're interested in
tickers = ['JPM', 'GS', 'MS', 'BLK', 'C']

# Download historical data for these assets until the end of November 2023
end_date = '2023-11-30'
data = yf.download(tickers, end=end_date)

# Preprocess the data by ensuring there are no missing values
data = data.dropna()

# Display the first few rows of the data
print(data.head())

# Plot the closing prices of the assets
plt.figure(figsize=(14, 7))
for ticker in tickers:
    plt.plot(data['Adj Close'][ticker], label=ticker)
plt.title('Adjusted Closing Prices')
plt.xlabel('Date')
plt.ylabel('Adjusted Price')
plt.legend()

class MovingAverageCrossoverStrategy:
    def __init__(self, short_window, long_window):
        self.short_window = short_window
        self.long_window = long_window

    def generate_signals(self, historical_prices):
        signals = pd.DataFrame(index=historical_prices.index)
        signals['signal'] = 0.0

        # Create short simple moving average over the short window
        signals['short_mavg'] = historical_prices.rolling(window=self.short_window, min_periods=1).mean()

        # Create long simple moving average over the long window
        signals['long_mavg'] = historical_prices.rolling(window=self.long_window, min_periods=1).mean()

        # Create signals
        signals['signal'][self.short_window:] = np.where(signals['short_mavg'][self.short_window:]
                                                         > signals['long_mavg'][self.short_window:], 1.0, 0.0)

        # Generate trading orders
        signals['positions'] = signals['signal'].diff()

        return signals

# Initialize the strategy with a short window of 40 days and a long window of 100 days
strategy = MovingAverageCrossoverStrategy(short_window=40, long_window=100)

# Generate trading signals for JPM
jpm_signals = strategy.generate_signals(data['Adj Close']['JPM'])

# Plot the signals along with the closing prices
plt.figure(figsize=(14, 7))
plt.plot(data['Adj Close']['JPM'], label='JPM Price', alpha=0.5)
plt.plot(jpm_signals['short_mavg'], label='40-day SMA', alpha=0.5)
plt.plot(jpm_signals['long_mavg'], label='100-day SMA', alpha=0.5)
plt.plot(jpm_signals.loc[jpm_signals.positions == 1.0].index,
         jpm_signals.short_mavg[jpm_signals.positions == 1.0],
         '^', markersize=10, color='m', label='Buy Signal')
plt.plot(jpm_signals.loc[jpm_signals.positions == -1.0].index,
         jpm_signals.short_mavg[jpm_signals.positions == -1.0],
         'v', markersize=10, color='k', label='Sell Signal')
plt.title('JPM Moving Average Crossover Strategy')
plt.xlabel('Date')
plt.ylabel('Adjusted Price')
plt.legend()

class Backtest:
    def __init__(self, initial_capital, historical_prices, signals):
        self.initial_capital = initial_capital
        self.historical_prices = historical_prices
        self.signals = signals
        self.positions = self.generate_positions()
        self.portfolio = self.backtest_portfolio()

    def generate_positions(self):
        positions = pd.DataFrame(index=self.signals.index).fillna(0.0)
        positions['JPM'] = 100 * self.signals['signal']   # This strategy buys 100 shares
        return positions

    def backtest_portfolio(self):
        portfolio = self.positions.multiply(self.historical_prices, axis=0)
        pos_diff = self.positions.diff()

        portfolio['holdings'] = (self.positions.multiply(self.historical_prices, axis=0)).sum(axis=1)
        portfolio['cash'] = self.initial_capital - (pos_diff.multiply(self.historical_prices, axis=0)).sum(axis=1).cumsum()

        portfolio['total'] = portfolio['cash'] + portfolio['holdings']
        portfolio['returns'] = portfolio['total'].pct_change()
        return portfolio

# Set the initial capital
initial_capital = float(100000.0)

# Create a Backtest instance
backtest = Backtest(initial_capital, data['Adj Close']['JPM'], jpm_signals)

# Plot the equity curve
plt.figure(figsize=(14, 7))
plt.plot(backtest.portfolio['total'], label='Portfolio value')
plt.title('Portfolio Value Over Time')
plt.xlabel('Date')
plt.ylabel('Portfolio Value in $')
plt.legend()

# Define the range of parameters to test
short_window_range = range(20, 60, 5)
long_window_range = range(80, 160, 10)

# Initialize variables to store the best parameters and performance
best_short_window = None
best_long_window = None
best_portfolio_value = -np.inf

# Grid search
for short_window in short_window_range:
    for long_window in long_window_range:
        if short_window >= long_window:
            continue

        # Initialize the strategy with the current set of parameters
        strategy = MovingAverageCrossoverStrategy(short_window, long_window)
        signals = strategy.generate_signals(data['Adj Close']['JPM'])

        # Backtest the strategy
        backtest = Backtest(initial_capital, data['Adj Close']['JPM'], signals)

        # Evaluate the strategy's performance
        final_portfolio_value = backtest.portfolio['total'].iloc[-1]

        # Update the best parameters if the current strategy is better
        if final_portfolio_value > best_portfolio_value:
            best_portfolio_value = final_portfolio_value
            best_short_window = short_window
            best_long_window = long_window

# Print the best parameters
print(f"Best Short Window: {best_short_window}")
print(f"Best Long Window: {best_long_window}")
print(f"Best Portfolio Value: {best_portfolio_value}")

Code based on https://theaiquant.medium.com/mastering-algorithmic-trading-crafting-strategies-from-concept-to-execution-e4adaf0c187f